In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.transforms import ToTensor
from pandas import Series
import numpy as np
from sklearn import preprocessing

In [ ]:
#The dataset, collected on the Marconi 100 system, has been published and
#described in the Scientific dataset publication: https://www.nature.com/articles/s41597-023-02174-3
#It is available on Zenodo: https://zenodo.org/records/7541722


#This is rack 1 of the dataset
!wget https://zenodo.org/record/7541722/files/1.tar?download=1 -o rack.tar

In [ ]:
!tar -xf /content/1.tar?download=1

In [ ]:
#This is node 36, of rack 1
file_name = '36.parquet'

In [ ]:
DATA = pd.read_parquet(file_name)
DATA.head(5)

,timestamp,ambient_avg,ambient_std,ambient_min,ambient_max,dimm0_temp_avg,dimm0_temp_std,dimm0_temp_min,dimm0_temp_max,dimm10_temp_avg,...,ps1_output_curre_max,ps1_output_volta_avg,ps1_output_volta_std,ps1_output_volta_min,ps1_output_volta_max,total_power_avg,total_power_std,total_power_min,total_power_max,value
0,2020-03-09 12:00:00+00:00,21.636362,0.077138,21.600000,21.799999,29.0,0.0,29.0,29.0,30.0,...,22.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
1,2020-03-09 12:15:00+00:00,21.860000,0.128063,21.600000,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399998,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
2,2020-03-09 12:30:00+00:00,21.885714,0.098975,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
3,2020-03-09 12:45:00+00:00,21.918917,0.098194,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,20.0,12.399999,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>
4,2020-03-09 13:00:00+00:00,21.875553,0.096967,21.799999,22.000000,29.0,0.0,29.0,29.0,30.0,...,21.0,12.400001,0.0,12.4,12.4,400.0,0.0,400.0,400.0,<NA>


In [ ]:
#The dataset has periods of unavailability of Nagios traces, as described in the dataset paper:  https://www.nature.com/articles/s41597-023-02174-3/figures/3
#We suggest dropping the periods where the labels are largely unavailable and using the dataset either after 1.4.2021 or 1.10.2021.
DATA = DATA[DATA['timestamp'] > '2021-04-01']

### parisa change
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)
## parisa change

<ipython-input-7-07ca0595a954>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['value'] = DATA['value'].replace(2,1)
<ipython-input-7-07ca0595a954>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['value'] = DATA['value'].replace(3,1)


In [ ]:
DATA.reset_index(drop=True, inplace = True)
DATA = DATA.fillna(0)
DATA['timestamp'] = pd.to_datetime(DATA['timestamp'])
l = DATA.timestamp.diff() == pd.Timedelta(minutes=15) #time consistency -> measurements should be 15 minutes apart
chunks = []
current_chunk = []
for index, value in enumerate(l):
    current_chunk.append(DATA.iloc[index])
    if not value:
        chunks.append(pd.DataFrame(current_chunk))
        current_chunk = []


if current_chunk:
    chunks.append(pd.DataFrame(current_chunk))

In [ ]:
revelent_len = [len(c) for c in chunks] ### pak she

In [ ]:
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change
relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
revelent_len = [len(c) for c in chunks if len(c) >= 20]
DATA2 = pd.concat(relevant)
DATA = DATA2
DATA['value'] = DATA['value'].replace(2,1)
DATA['value'] = DATA['value'].replace(3,1)

### parisa change end
##3 parisa change
# relevant = [c for c in chunks if len(c) >= 20] #assuming time window of 20
# DATA2 = pd.concat(relevant)
# DATA = DATA2
# DATA['value'] = DATA['value'].replace(2,1)
# DATA['value'] = DATA['value'].replace(3,1)
# DATA = DATA.drop(columns=['timestamp'])
# DATA = DATA.astype(float)
##3 parisa change end

In [ ]:
DATA3= DATA
DATA=DATA.drop(columns=['timestamp'])
DATA = DATA.astype(float)

In [ ]:
scaler = preprocessing.MinMaxScaler()
names = DATA.columns
d = scaler.fit_transform(DATA)
DATA = pd.DataFrame(d, columns=names)
### parisa change
# DATA['timestamp'] = DATA3['timestamp'].astype('int64') // 10**9
timestamp_scaler = preprocessing.MinMaxScaler()
DATA['timestamp'] = timestamp_scaler.fit_transform(DATA3['timestamp'].astype('int64').values.reshape(-1, 1))
#### parisa change end

In [ ]:
train_data = DATA[:int(DATA.shape[0]*0.8)]
test_data = DATA[int(DATA.shape[0]*0.8):]

In [ ]:
train_data.shape,test_data.shape

((41847, 354), (10462, 354))

In [ ]:
train_data.shape

(41847, 354)

In [ ]:
train_data['value'].value_counts()

,count
value,
0.0,40673
1.0,1174


In [ ]:
train_feat = train_data.drop(columns=['value'])
train_feat = train_feat.to_numpy()
test_feat = test_data.drop(columns=['value'])
test_feat = test_feat.to_numpy()

In [ ]:
train_feat = torch.tensor(train_feat, dtype=torch.float)
test_feat = torch.tensor(test_feat, dtype=torch.float)

In [ ]:
test_labels = test_data['value']
test_labels = test_labels.to_numpy()
test_labels = torch.tensor(test_labels, dtype=torch.float)
test_labels

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_window = 20 ## 20

def create_data_seq(data,tw):
    seq = []
    for i in range(len(data)-tw):
        x_seq = data[i:i+tw],
        y_seq = data[i+tw:i+tw+1]  ###در ترین میاد 0 تا 20 رو میده و میگه 21 رو حدس بزن
        seq.append((x_seq,y_seq))
    return seq
### parisa change
# def create_data_seq(data, tw, relevant_len):
#     seq = []
#     start_idx = 0

#     for length in relevant_len:
#         end_idx = start_idx + length

#         for i in range(start_idx, end_idx):
#             # Determine the actual window size
#             actual_tw = min(tw, length)

#             if i + actual_tw >= end_idx:
#                 break

#             # Create x_seq and y_seq
#             x_seq = data[i:i+actual_tw].unsqueeze(0)  # Add batch dimension here
#             y_seq = data[i+actual_tw:i+actual_tw+1]

#             if len(y_seq) > 0:
#                 seq.append((x_seq, y_seq))

#         start_idx = end_idx

#     return seq

## parisa change end

In [ ]:
# train_seq = create_data_seq(train_feat,train_window,revelent_len) ## parisa change
train_seq = create_data_seq(train_feat,train_window) ## parisa change

In [ ]:
X[0].size()


torch.Size([1, 20, 353])

In [ ]:
train_loader = DataLoader(train_seq, batch_size=1, shuffle=True)

X,y  = next(iter(train_loader))
print(X)


[tensor([[[0.5593, 0.0174, 0.5896,  ..., 0.3919, 0.7917, 0.0394],
         [0.5571, 0.0159, 0.5896,  ..., 0.3919, 0.8333, 0.0394],
         [0.5579, 0.0220, 0.5849,  ..., 0.3919, 0.7708, 0.0395],
         ...,
         [0.5607, 0.0056, 0.5943,  ..., 0.3784, 0.8438, 0.0397],
         [0.5571, 0.0196, 0.5849,  ..., 0.3649, 0.7500, 0.0398],
         [0.5554, 0.0139, 0.5849,  ..., 0.3649, 0.7708, 0.0398]]])]


In [ ]:
X,y = next(iter(train_loader))
print(X)
print(y)

[tensor([[[0.4983, 0.0000, 0.5283,  ..., 0.3514, 0.4479, 0.4595],
         [0.4982, 0.0056, 0.5236,  ..., 0.3378, 0.6667, 0.4595],
         [0.4975, 0.0146, 0.5236,  ..., 0.3514, 0.8333, 0.4595],
         ...,
         [0.4983, 0.0000, 0.5283,  ..., 0.3514, 0.8125, 0.4596],
         [0.4983, 0.0000, 0.5283,  ..., 0.3378, 0.8125, 0.4597],
         [0.4978, 0.0120, 0.5236,  ..., 0.3514, 0.7812, 0.4597]]])]
tensor([[[0.4983, 0.0000, 0.5283, 0.4628, 0.6643, 0.0713, 0.6889, 0.6531,
          0.6485, 0.0000, 0.6739, 0.6200, 0.6308, 0.0000, 0.6522, 0.6000,
          0.6393, 0.0000, 0.6596, 0.6078, 0.6452, 0.0000, 0.6809, 0.6154,
          0.6279, 0.0000, 0.6522, 0.6000, 0.6555, 0.1578, 0.6667, 0.6346,
          0.6766, 0.1540, 0.6889, 0.6400, 0.6678, 0.1450, 0.6889, 0.6400,
          0.6766, 0.1529, 0.6889, 0.6400, 0.6574, 0.0000, 0.6889, 0.6200,
          0.6605, 0.0000, 0.6889, 0.6200, 0.6555, 0.0000, 0.6739, 0.6200,
          0.6545, 0.1720, 0.6667, 0.6327, 0.6347, 0.0000, 0.6522, 0.6000,


In [ ]:
# test_seq = create_data_seq(test_feat,train_window,revelent_len)
test_seq = create_data_seq(test_feat,train_window)

In [ ]:
test_loader = DataLoader(test_seq, batch_size=1, shuffle=False) ## در هر فور ما یه تعدادی بچ داریم که میدیم به دل با بچ بیشتر میتونیم بگیم مثلا 2 تا 2 تا بهش بده
X,y = next(iter(test_loader))

In [ ]:
class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=353,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=8,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x[:, :, :]

In [ ]:
############### mine
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=353,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x[:, :, :]

In [ ]:
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )
        self.lstm2 = torch.nn.LSTM(
          input_size=16,
          hidden_size=353,
          num_layers=1,
          batch_first=True
        )

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)

        x, (_, _) = self.lstm1(x, (h0, c0))

        h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))
        return x

In [ ]:
### mine
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=353,
            batch_first=True,
            num_layers=1
        )

    def forward(self, x):
        batch_size = x.shape[0]

        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        x, (_, _) = self.lstm1(x, (h0, c0))

        # Initial hidden and cell states for lstm2
        h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        x, (_, _) = self.lstm2(x, (h0, c0))

        # Initial hidden and cell states for lstm3
        h0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        c0 = torch.zeros(1, batch_size, 353).requires_grad_().to(device)
        x, (_, _) = self.lstm3(x, (h0, c0))

        return x

In [ ]:
############### mine for add h,c
# class encoder(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.lstm1 = torch.nn.LSTM(
#             input_size=352,
#             hidden_size=32,
#             batch_first=True,
#             num_layers=1
#         )
#         self.lstm2 = torch.nn.LSTM(
#           input_size=32,
#           hidden_size=8,
#           num_layers=1,
#           batch_first=True
#         )

#     def forward(self, x):
#         batch_size = x.shape[0]
#         h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)

#         x, (_, _) = self.lstm1(x, (h0, c0))

#         h0 = torch.zeros(1, batch_size,8).requires_grad_().to(device)
#         c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
#         x, (_, _) = self.lstm2(x, (h0, c0))
#         return x[:, :, :]

class encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer
        self.lstm1 = torch.nn.LSTM(
            input_size=353,
            hidden_size=32,  # Increased the hidden size for this example
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=32,   # Input size matches the hidden size of lstm1
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer (newly added)
        self.lstm3 = torch.nn.LSTM(
            input_size=16,   # Input size matches the hidden size of lstm2
            hidden_size=8,
            batch_first=True,
            num_layers=1
        )
        self.h_proj_1_to_2 = torch.nn.Linear(32, 16)
        self.c_proj_1_to_2 = torch.nn.Linear(32, 16)

        self.h_proj_2_to_3 = torch.nn.Linear(16, 8)
        self.c_proj_2_to_3 = torch.nn.Linear(16, 8)

    def forward(self, x):
        batch_size = x.shape[0]
        # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 8).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))

        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        # Initial hidden and cell states for lstm1
        h0 = torch.zeros(1, batch_size, 32).to(x.device)
        c0 = torch.zeros(1, batch_size, 32).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h0, c0))

        # Resize h1, c1 to match the input size of lstm2
        h1 = self.h_proj_1_to_2(h1)
        c1 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h1, c1))

        # Resize h2, c2 to match the input size of lstm3
        h2 = self.h_proj_2_to_3(h2)
        c2 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h2, c2))
        return x[:, :, :]

In [ ]:
### mine for add h,c
class decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # First LSTM layer (added for symmetry with the encoder)
        self.lstm1 = torch.nn.LSTM(
            input_size=8,
            hidden_size=16,
            batch_first=True,
            num_layers=1
        )

        # Second LSTM layer
        self.lstm2 = torch.nn.LSTM(
            input_size=16,
            hidden_size=32,
            batch_first=True,
            num_layers=1
        )

        # Third LSTM layer
        self.lstm3 = torch.nn.LSTM(
            input_size=32,
            hidden_size=353,
            batch_first=True,
            num_layers=1
        )

        self.h_proj_1_to_2 = torch.nn.Linear(16, 32)
        self.c_proj_1_to_2 = torch.nn.Linear(16, 32)

        self.h_proj_2_to_3 = torch.nn.Linear(32, 353)
        self.c_proj_2_to_3 = torch.nn.Linear(32, 353)

    def forward(self, x):
        batch_size = x.shape[0]

        # # Initial hidden and cell states for lstm1
        # h0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 16).requires_grad_().to(device)
        # x, (_, _) = self.lstm1(x, (h0, c0))

        # # Initial hidden and cell states for lstm2
        # h0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 32).requires_grad_().to(device)
        # x, (_, _) = self.lstm2(x, (h0, c0))

        # # Initial hidden and cell states for lstm3
        # h0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # c0 = torch.zeros(1, batch_size, 352).requires_grad_().to(device)
        # x, (_, _) = self.lstm3(x, (h0, c0))


        ###### parisa changes of model give h , c to new layaer

        # Initial hidden and cell states for lstm1
        h1 = torch.zeros(1, batch_size, 16).to(x.device)
        c1 = torch.zeros(1, batch_size, 16).to(x.device)
        x, (h1, c1) = self.lstm1(x, (h1, c1))

        # Resize h1, c1 to match the input size of lstm2
        h2 = self.h_proj_1_to_2(h1)
        c2 = self.c_proj_1_to_2(c1)
        x, (h2, c2) = self.lstm2(x, (h2, c2))

        # Resize h2, c2 to match the input size of lstm3
        h3 = self.h_proj_2_to_3(h2)
        c3 = self.c_proj_2_to_3(c2)
        x, (h3, c3) = self.lstm3(x, (h3, c3))

        return x

In [ ]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.decoder = decoder().to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8,16)
        self.fc2 = torch.nn.Linear(16,353)


    def forward(self, x):
        x = self.encoder(x)
        #print('LSTM out', x.shape)
        x = self.fc1(x)
        #print('DENSE 1', x.shape)
        x = self.fc2(x)
        #print('DENSE 2', x.shape)

        return x

In [ ]:
#### mine
# class RUAD(torch.nn.Module):
#     def __init__(self):
#         super().__init__()

#         self.encoder = encoder().to(device)
#         self.fc1 = torch.nn.Linear(8,16)
#         self.fc2 = torch.nn.Linear(16,352)


#     def forward(self, x):
#         x = self.encoder(x)
#         #print('LSTM out', x.shape)
#         x = self.fc1(x)
#         #print('DENSE 1', x.shape)
#         x = self.fc2(x)
#         #print('DENSE 2', x.shape)

#         return x

In [ ]:
### mine
class RUAD(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = encoder().to(device)
        self.fc1 = torch.nn.Linear(8, 16)
        self.fc2 = torch.nn.Linear(16, 8)
        self.decoder = decoder().to(device)  # Adding the decoder to RUAD

    def forward(self, x):
        x = self.encoder(x)
        # Passing through the fully connected layers   ----- new changes
        # x = self.fc1(x)
        # x = self.fc2(x)
        # Passing the result through the decoder ------- end of new changes
        x = self.decoder(x)

        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = RUAD()
model

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  ### adam ro bezar va bebin
criterion = torch.nn.MSELoss()

In [ ]:
for d in train_loader:
    print(len(d[0]))
    print(d[0][0].size())
    print(d[1].size())
    break
    ## batch size,(row count,column count) --> sequence

1
torch.Size([1, 20, 353])
torch.Size([1, 1, 353])


In [ ]:
print(model)

RUAD(
  (encoder): encoder(
    (lstm1): LSTM(353, 32, batch_first=True)
    (lstm2): LSTM(32, 16, batch_first=True)
    (lstm3): LSTM(16, 8, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (c_proj_1_to_2): Linear(in_features=32, out_features=16, bias=True)
    (h_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
    (c_proj_2_to_3): Linear(in_features=16, out_features=8, bias=True)
  )
  (fc1): Linear(in_features=8, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=8, bias=True)
  (decoder): decoder(
    (lstm1): LSTM(8, 16, batch_first=True)
    (lstm2): LSTM(16, 32, batch_first=True)
    (lstm3): LSTM(32, 353, batch_first=True)
    (h_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (c_proj_1_to_2): Linear(in_features=16, out_features=32, bias=True)
    (h_proj_2_to_3): Linear(in_features=32, out_features=353, bias=True)
    (c_proj_2_to_3): Linear(in_features=32, out_features=353,

In [ ]:
%%time
def train():
    model.train()
    for d in train_loader:
        #print(type(d))
        x = d[0][0].to(device)
        y = d[1].to(device)
        optimizer.zero_grad()
        out = model(x)
        # print(out[0][9])
        loss = criterion(out[0][train_window-1], y[0][0])
        loss.backward()
        optimizer.step()
    return float(loss)

for epoch in range(6):
    loss = train()
    print(f'Epoch: {epoch+1:02d}, Loss: {loss:.4f}')




############ results
########### result change layer 8 to 32
# Epoch: 01, Loss: 0.0188
# Epoch: 02, Loss: 0.0273
# Epoch: 03, Loss: 0.0033
# CPU times: user 6min 3s, sys: 3.62 s, total: 6min 6s
# Wall time: 6min 12s

########### result add hidden ayer 32
# Epoch: 01, Loss: 0.0021
# Epoch: 02, Loss: 0.0378
# Epoch: 03, Loss: 0.0069
# CPU times: user 7min 58s, sys: 4.28 s, total: 8min 2s
# Wall time: 8min 11s

########### result add hidden layer 32 to decoder too and add decoder to ruad model with gpu
# Epoch: 01, Loss: 0.0200
# Epoch: 02, Loss: 0.0210
# Epoch: 03, Loss: 0.0231
# CPU times: user 13min 7s, sys: 14.9 s, total: 13min 22s
# Wall time: 13min 5s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu
# Epoch: 01, Loss: 0.0089
# Epoch: 02, Loss: 0.0483
# Epoch: 03, Loss: 0.0035
# CPU times: user 12min 38s, sys: 13.5 s, total: 12min 51s
# Wall time: 12min 34s

########### result first ruad model with gpu and add timestamp column and window 10 and seq
# Epoch: 01, Loss: 0.0107
# Epoch: 02, Loss: 0.0098
# Epoch: 03, Loss: 0.0219
# CPU times: user 11min 17s, sys: 4.6 s, total: 11min 22s
# Wall time: 11min 28s

########### result first ruad model with gpu and add timestamp column and window 10 and remove seq
# Epoch: 01, Loss: 0.0252
# Epoch: 02, Loss: 0.0093
# Epoch: 03, Loss: 0.0188
# CPU times: user 11min 8s, sys: 4.67 s, total: 11min 12s
# Wall time: 11min 19s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , adam optimizer
# Epoch: 01, Loss: 0.0365
# Epoch: 02, Loss: 0.0473
# Epoch: 03, Loss: 0.0536
# CPU times: user 12min 14s, sys: 13.8 s, total: 12min 28s
# Wall time: 12min 12s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer
# Epoch: 01, Loss: 0.0594
# Epoch: 02, Loss: 0.0260
# Epoch: 03, Loss: 0.0311
# CPU times: user 11min 27s, sys: 13.3 s, total: 11min 40s
# Wall time: 11min 24s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 8
# Epoch: 01, Loss: 0.0270
# Epoch: 02, Loss: 0.0267
# Epoch: 03, Loss: 0.0317
# Epoch: 04, Loss: 0.0313
# Epoch: 05, Loss: 0.0615
# CPU times: user 19min 21s, sys: 23.7 s, total: 19min 44s
# Wall time: 19min 18s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 5 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# CPU times: user 20min 12s, sys: 12 s, total: 20min 24s
# Wall time: 20min 39s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0211
# Epoch: 02, Loss: 0.0153
# Epoch: 03, Loss: 0.0244
# Epoch: 04, Loss: 0.0031
# Epoch: 05, Loss: 0.0129
# Epoch: 06, Loss: 0.0030
# CPU times: user 24min 16s, sys: 14.37 s, total: 24min 30s
# Wall time: 24min 48s

########### result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq , SGD optimizer, epoch 6 , add h,c
# Epoch: 01, Loss: 0.0042
# Epoch: 02, Loss: 0.0035
# Epoch: 03, Loss: 0.0061
# Epoch: 04, Loss: 0.0053
# Epoch: 05, Loss: 0.0214
# Epoch: 06, Loss: 0.0109
# CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
# Wall time: 26min 46s


Epoch: 01, Loss: 0.0042
Epoch: 02, Loss: 0.0035
Epoch: 03, Loss: 0.0061
Epoch: 04, Loss: 0.0053
Epoch: 05, Loss: 0.0214
Epoch: 06, Loss: 0.0109
CPU times: user 26min 13s, sys: 14.2 s, total: 26min 27s
Wall time: 26min 46s


In [ ]:
def test(x):
    model.eval()
    x = x.to(device)
    out = model(x)
    return out

In [ ]:
X, y = next(iter(test_loader))
#print(X.shape)
print(len(X[0][0]))

10


In [ ]:
out = test(X[0])

In [ ]:
for d in test_loader:
    print(d[0][0].size())
    break

torch.Size([1, 10, 353])


In [ ]:
for d in test_loader:
    x = d[0][0].to(device)
    y = d[1].to(device)
    out = model(x)
    break

In [ ]:
from sklearn.metrics import mean_squared_error,roc_auc_score
reconstruction = []
loss = []
#y_true = []
for d in test_loader:
    x = d[0][0]
    y = d[1]
    out = test(x)
    reconstruction.append(out)
    loss.append(mean_squared_error(out[0][train_window-1].detach().cpu().numpy(),y[0][0]))
    #y_true.append(d[1].detach().cpu().numpy())

In [ ]:
# test_seq_label = create_data_seq(test_labels,train_window,revelent_len)
test_seq_label = create_data_seq(test_labels,train_window)

In [ ]:
temp_list = []
for i in range(len(test_seq_label)):
    temp = test_seq_label[i][1].type(torch.int64)
    temp = temp.detach().cpu().numpy().tolist()
    temp_list.append(temp)

In [ ]:
y_true = []
for i in range(len(temp_list)):
    y_true.append(temp_list[i][0])

In [ ]:
error_df = pd.DataFrame({'loss': loss,
                        'true_class': y_true})
error_df.describe()

,loss,true_class
count,10442.000000,10442.000000
mean,0.016721,0.013120
std,0.039142,0.113795
min,0.002611,0.000000
25%,0.006026,0.000000
50%,0.008819,0.000000
75%,0.015921,0.000000
max,0.295949,1.000000


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(error_df['true_class'],error_df['loss'])

0.9069957536027086

In [ ]:
#0.8765328998395648
#0.8723729179726375    result of change hidden layer from 8 to 32
#0.9065247187071686    result of add one layer 32 between encoder :)
######0.904416749009233     result add hidden layer 32 to decoder too and add decoder to ruad model with gpu (this is pretty good)   ----- moshtrak
#####0.8655186444515994    result first ruad model with gpu and add timestamp column and woindow 10 and seq , adam optimizer
#0.8967381497429511    result first ruad model with gpu and add timestamp column and woindow 10 and remove seq , adam optimizer
#0.3060959342747257    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, adam optimizer
#0.34766214604908874   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer
#0.36209899126422795   result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 +3 (i forgot to reset)
#0.907213292243243     result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 5 , add , h,c
#0.9098181020482538    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 10 and remove seq, SGD optimizer , epoch 6 , add , h,c
#0.9069957536027086    result add hidden layer 32 to decoder too and add decoder to ruad model and remove fully connected layers with gpu window 20 and remove seq, SGD optimizer , epoch 6 , add , h,c